In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nurulhadiiiiiiiii","key":"1a473a0f6ec5eb1b87203db7ba466395"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d rashikrahmanpritom/heart-attack-analysis-prediction-dataset

Dataset URL: https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset
License(s): CC0-1.0
  0% 0.00/4.11k [00:00<?, ?B/s]
100% 4.11k/4.11k [00:00<00:00, 12.4MB/s]


In [4]:
!mkdir heart-attack-analysis-prediction-dataset
!unzip heart-attack-analysis-prediction-dataset -d heart-attack-analysis-prediction-dataset
!ls heart-attack-analysis-prediction-dataset

Archive:  heart-attack-analysis-prediction-dataset.zip
  inflating: heart-attack-analysis-prediction-dataset/heart.csv  
  inflating: heart-attack-analysis-prediction-dataset/o2Saturation.csv  
heart.csv  o2Saturation.csv


In [54]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import plot_model

In [55]:
df = pd.read_csv('/content/heart-attack-analysis-prediction-dataset/heart.csv')
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [56]:
X = df.drop(['output'], axis=1)
y = df[['output']]

In [57]:
sb = MinMaxScaler()
new_x = sb.fit_transform(X)

In [58]:
x_train_1, x_test, y_train_1, y_test = train_test_split(new_x, y, test_size=0.1)
x_train, x_validation, y_train, y_validation = train_test_split(x_train_1, y_train_1, test_size = 0.1)

In [59]:
model = Sequential([
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [60]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [61]:
history = model.fit(x_train, y_train, epochs=50)

Epoch 1/50
8/8 [==============================] - 2s 4ms/step - loss: 0.6487 - accuracy: 0.6885
Epoch 2/50
8/8 [==============================] - 0s 4ms/step - loss: 0.5157 - accuracy: 0.7869
Epoch 3/50
8/8 [==============================] - 0s 4ms/step - loss: 0.4187 - accuracy: 0.8238
Epoch 4/50
8/8 [==============================] - 0s 4ms/step - loss: 0.3808 - accuracy: 0.8566
Epoch 5/50
8/8 [==============================] - 0s 4ms/step - loss: 0.4078 - accuracy: 0.8279
Epoch 6/50
8/8 [==============================] - 0s 4ms/step - loss: 0.4134 - accuracy: 0.8402
Epoch 7/50
8/8 [==============================] - 0s 4ms/step - loss: 0.4203 - accuracy: 0.8279
Epoch 8/50
8/8 [==============================] - 0s 4ms/step - loss: 0.4195 - accuracy: 0.8033
Epoch 9/50
8/8 [==============================] - 0s 4ms/step - loss: 0.3645 - accuracy: 0.8443
Epoch 10/50
8/8 [==============================] - 0s 4ms/step - loss: 0.3325 - accuracy: 0.8648
Epoch 11/50
8/8 [======================

In [62]:
loss, accuracy = model.evaluate(x_test, y_test)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

1/1 [==============================] - 0s 123ms/step - loss: 0.3007 - accuracy: 0.9355
Loss: 0.30068427324295044
Accuracy: 0.9354838728904724


In [63]:
model.save('./uas.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [64]:
model = tf.keras.models.load_model('./uas.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./uas.tflite", "wb").write(tflite_model)

185960